In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from IPython.display import display

In [132]:
data = pd.read_csv("cleaned_copy1.csv")
data = data.drop(columns="Unnamed: 0")
data = data.drop(columns="Date")
data = data.rename(columns={"HT Red Cards.1":"AT Red Cards"})

data.head()

,Time,HomeTeam,AwayTeam,Home Goals,Away Goals,Winner,Home Team Shots,Away Team Shots,HT Shots on Target,AT Shots on Target,HT Fouls,AT Fouls,HT Corners,AT Corners,HT Yellow Cards,AT Yellow Cards,HT Red Cards,AT Red Cards
0,12:30,Fulham,Arsenal,0,3,A,5,13,2,6,12,12,2,3,2,2,0,0
1,15:00,Crystal Palace,Southampton,1,0,H,5,9,3,5,14,11,7,3,2,1,0,0
2,17:30,Liverpool,Leeds,4,3,H,22,6,6,3,9,6,9,0,1,0,0,0
3,20:00,West Ham,Newcastle,0,2,A,15,15,3,2,13,7,8,7,2,2,0,0
4,14:00,West Brom,Leicester,0,3,A,7,13,1,7,12,9,2,5,1,1,0,0


In [133]:
data.dtypes

Time                  object
HomeTeam              object
AwayTeam              object
Home Goals             int64
Away Goals             int64
Winner                object
Home Team Shots        int64
Away Team Shots        int64
HT Shots on Target     int64
AT Shots on Target     int64
HT Fouls               int64
AT Fouls               int64
HT Corners             int64
AT Corners             int64
HT Yellow Cards        int64
AT Yellow Cards        int64
HT Red Cards           int64
AT Red Cards           int64
dtype: object

In [134]:
HomeTeam_dummies = pd.get_dummies(data["HomeTeam"])
AwayTeam_dummies = pd.get_dummies(data["AwayTeam"])

data = pd.concat([data, HomeTeam_dummies], axis=1)
data = pd.concat([data, AwayTeam_dummies], axis=1)

# Drop the original education column
data = data.drop(columns=["HomeTeam", "AwayTeam"])


In [135]:
data["hour"] = data["Time"].str.replace(":.+", "", regex=True).astype("int")
data = data.drop(columns=["Time"])

In [136]:
def encode_Winner(Winner):

    if Winner == "H":
        return 1
    else:
        return 0

data["Winner"] = data["Winner"].apply(encode_Winner)

In [137]:
data.head()

,Home Goals,Away Goals,Winner,Home Team Shots,Away Team Shots,HT Shots on Target,AT Shots on Target,HT Fouls,AT Fouls,HT Corners,...,Man City,Man United,Newcastle,Sheffield United,Southampton,Tottenham,West Brom,West Ham,Wolves,hour
0,0,3,0,5,13,2,6,12,12,2,...,0,0,0,0,0,0,0,0,0,12
1,1,0,1,5,9,3,5,14,11,7,...,0,0,0,0,1,0,0,0,0,15
2,4,3,1,22,6,6,3,9,6,9,...,0,0,0,0,0,0,0,0,0,17
3,0,2,0,15,15,3,2,13,7,8,...,0,0,1,0,0,0,0,0,0,20
4,0,3,0,7,13,1,7,12,9,2,...,0,0,0,0,0,0,0,0,0,14


In [138]:
data.dtypes

Home Goals            int64
Away Goals            int64
Winner                int64
Home Team Shots       int64
Away Team Shots       int64
HT Shots on Target    int64
AT Shots on Target    int64
HT Fouls              int64
AT Fouls              int64
HT Corners            int64
AT Corners            int64
HT Yellow Cards       int64
AT Yellow Cards       int64
HT Red Cards          int64
AT Red Cards          int64
Arsenal               uint8
Aston Villa           uint8
Brighton              uint8
Burnley               uint8
Chelsea               uint8
Crystal Palace        uint8
Everton               uint8
Fulham                uint8
Leeds                 uint8
Leicester             uint8
Liverpool             uint8
Man City              uint8
Man United            uint8
Newcastle             uint8
Sheffield United      uint8
Southampton           uint8
Tottenham             uint8
West Brom             uint8
West Ham              uint8
Wolves                uint8
Arsenal             

In [139]:
list(data)

['Home Goals',
 'Away Goals',
 'Winner',
 'Home Team Shots',
 'Away Team Shots',
 'HT Shots on Target',
 'AT Shots on Target',
 'HT Fouls',
 'AT Fouls',
 'HT Corners',
 'AT Corners',
 'HT Yellow Cards',
 'AT Yellow Cards',
 'HT Red Cards',
 'AT Red Cards',
 'Arsenal',
 'Aston Villa',
 'Brighton',
 'Burnley',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Fulham',
 'Leeds',
 'Leicester',
 'Liverpool',
 'Man City',
 'Man United',
 'Newcastle',
 'Sheffield United',
 'Southampton',
 'Tottenham',
 'West Brom',
 'West Ham',
 'Wolves',
 'Arsenal',
 'Aston Villa',
 'Brighton',
 'Burnley',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Fulham',
 'Leeds',
 'Leicester',
 'Liverpool',
 'Man City',
 'Man United',
 'Newcastle',
 'Sheffield United',
 'Southampton',
 'Tottenham',
 'West Brom',
 'West Ham',
 'Wolves',
 'hour']

In [140]:
from sklearn.preprocessing import StandardScaler

data_scaled = StandardScaler().fit_transform(data[['Home Goals',
 'Away Goals',
 'Home Team Shots',
 'Away Team Shots',
 'HT Shots on Target',
 'AT Shots on Target',
 'HT Fouls',
 'AT Fouls',
 'HT Corners',
 'AT Corners',
 'HT Yellow Cards',
 'AT Yellow Cards',
 'HT Red Cards',
 'AT Red Cards', 
'hour']])

data_scaled

array([[-1.02577852,  1.31991093, -1.42539275, ..., -0.22941573,
        -0.24843938, -1.7438882 ],
       [-0.26742086, -1.06849933, -1.42539275, ..., -0.22941573,
        -0.24843938, -0.67101581],
       [ 2.00765212,  1.31991093,  1.67495647, ..., -0.22941573,
        -0.24843938,  0.04423246],
       ...,
       [-0.26742086, -1.06849933, -0.14877837, ..., -0.22941573,
        -0.24843938, -0.31339167],
       [ 1.24929446, -1.06849933,  0.2159686 , ..., -0.22941573,
        -0.24843938, -0.31339167],
       [-0.26742086,  0.52377418,  0.2159686 , ..., -0.22941573,
        -0.24843938, -0.31339167]])

In [141]:
# Create a DataFrame of the scaled data
data_scaled = pd.DataFrame(data_scaled, columns=['Home Goals',
 'Away Goals',
 'Home Team Shots',
 'Away Team Shots',
 'HT Shots on Target',
 'AT Shots on Target',
 'HT Fouls',
 'AT Fouls',
 'HT Corners',
 'AT Corners',
 'HT Yellow Cards',
 'AT Yellow Cards',
 'HT Red Cards',
 'AT Red Cards',
'hour'])

# Replace the original data with the columns of information from the scaled Data
data["Home Goals"] = data_scaled["Home Goals"]
data["Away Goals"] = data_scaled["Away Goals"]
data['Home Team Shots'] = data_scaled['Home Team Shots']
data['Home Team Shots'] = data_scaled['Home Team Shots']
data['Away Team Shots'] = data_scaled['Away Team Shots']
data['HT Shots on Target'] = data_scaled['HT Shots on Target']
data['AT Shots on Target'] = data_scaled['AT Shots on Target']
data['HT Fouls'] = data_scaled['HT Fouls']
data['AT Fouls'] = data_scaled['AT Fouls']
data['HT Corners'] = data_scaled['HT Corners']
data['AT Corners'] = data_scaled['AT Corners']
data['HT Yellow Cards'] = data_scaled['HT Yellow Cards']
data['AT Yellow Cards'] = data_scaled['AT Yellow Cards']
data['HT Red Cards'] = data_scaled['HT Red Cards']
data['AT Red Cards'] = data_scaled['AT Red Cards']
data['hour'] = data_scaled['hour']

# Review the DataFrame
data.head()

,Home Goals,Away Goals,Winner,Home Team Shots,Away Team Shots,HT Shots on Target,AT Shots on Target,HT Fouls,AT Fouls,HT Corners,...,Man City,Man United,Newcastle,Sheffield United,Southampton,Tottenham,West Brom,West Ham,Wolves,hour
0,-1.025779,1.319911,0,-1.425393,0.335819,-0.982300,0.849355,0.226095,0.417844,-1.172044,...,0,0,0,0,0,0,0,0,0,-1.743888
1,-0.267421,-1.068499,1,-1.425393,-0.484832,-0.596288,0.406010,0.808579,0.129676,0.475059,...,0,0,0,0,1,0,0,0,0,-0.671016
2,2.007652,1.319911,1,1.674956,-1.100321,0.561750,-0.480679,-0.647629,-1.311166,1.133900,...,0,0,0,0,0,0,0,0,0,0.044232
3,-1.025779,0.523774,0,0.398342,0.746145,-0.596288,-0.924024,0.517337,-1.022998,0.804480,...,0,0,1,0,0,0,0,0,0,1.117105
4,-1.025779,1.319911,0,-1.060646,0.335819,-1.368313,1.292700,0.226095,-0.446661,-1.172044,...,0,0,0,0,0,0,0,0,0,-1.028640


In [153]:
data['Winner'].value_counts()

0    236
1    144
Name: Winner, dtype: int64

In [142]:
y = data["Winner"]
X = data.drop(["Winner"],1)

/var/folders/lt/xhwk7z354rl1rjbqlbct1dcm0000gn/T/ipykernel_30016/672423763.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = data.drop(["Winner"],1)


In [154]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=3, 
                                                    stratify=y)
X_train.shape 

(285, 55)

In [163]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn import svm

clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
scores.shape

(5,)

In [169]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

1.00 accuracy with a standard deviation of 0.00


In [164]:
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=2)
classifier

LogisticRegression(max_iter=200, random_state=2)

In [165]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=2)

In [166]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9894736842105263


In [167]:
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,1,1
1,0,0
2,0,0
3,1,1
4,0,0
5,1,1
6,0,0
7,0,0
8,0,0
9,0,0


In [168]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, predictions)

0.9894736842105263